強化学習の「GRPO」をCartPoleで実装しながら解説 のコードのPendulum版です
===================



**1. 概要**：

「DeepSeek-R1」で有名となった強化学習手法 GRPO（Group Relative Policy Optimization）を、CartPoleで実装しながら学びます

の、Pendulum版です





**2. 実装者**：小川雄太郎

**3. 実装日**：2025年02月08日

**4. 実行環境**：Google Colabratory、CPU

**5. 参考記事・スライド類**


ブログ: 「LLMチューニングのための強化学習：GRPO（Group Relative Policy Optimization）」 [[link](https://horomary.hatenablog.com/entry/2025/01/26/204545)]


**6. 実装のメイン参考**

ブログ: 「group relative policy optimization (GRPO)」 [[link](https://superb-makemake-3a4.notion.site/group-relative-policy-optimization-GRPO-18c41736f0fd806eb39dc35031758885)]


**7. その他参考にした記事**

論文

DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via Reinforcement Learning [[link](https://arxiv.org/abs/2501.12948)]

DeepSeekMath: Pushing the Limits of Mathematical Reasoning in Open Language Models [[link](https://arxiv.org/abs/2402.03300)]


**8. その他**

注釈: 筆者はDeepSeekの思想問題、コンプライアンス問題などについては、肯定的立場でも否定的立場でもありません。中立的立場でもありません。そこにある技術的側面のみに関心を持ち、取り挙げます

---

# [0] 初期設定


## [0-1] GPUの初期設定を確認


In [1]:
!nvidia-smi

Sat Feb  8 09:34:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## [0-2] package installの実施

In [2]:
# pass

## [0-3] importの実施

In [3]:
# 定番系
import os
import sys
import random
import math
import time
import re
import datetime
import json
import argparse
import numpy as np
import pandas as pd
from pandas.errors import EmptyDataError
from pandas.errors import ParserError
from PIL import Image
from tqdm import tqdm
import gc

# 描画系
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc
from ipywidgets import interact
%matplotlib inline

# PyTorch関連
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms

In [4]:
# warning を非表示にしてしまう
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# stderr を無効化
sys.stderr = open('/dev/null', 'w')


## [0-4] 乱数シードの初期化


In [5]:
# 乱数シードの初期化
seed_val = 1234

print("使用するseedの値：", str(seed_val))
os.environ['PYTHONHASHSEED'] = str(seed_val)
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val) # PyTorchはこちら参考 https://pytorch.org/docs/stable/notes/randomness.html
torch.cuda.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
# GPU高速化
torch.backends.cudnn.benchmark = True

# GPUの再現性まで確保したい場合（ただし、計算速度が低下してしまいます）
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False


使用するseedの値： 1234


## [0-5] 実行環境の確認

In [6]:
# -----------------------------
# 環境情報の確認
# -----------------------------
def print_program_info():
    print("Pythonのバージョン：", sys.version)
    print("---------")
    print("PyTorchのバージョン：",
    torch.__version__)
    print("GPUの枚数：", torch.cuda.device_count())
    global device  # 変数deviceは以降もプログラム本体で使用するのでglobal変数にしています
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)


# 【実行】確認
print_program_info()

# 上記で変数deviceをglobalにしており、deviceでGPUを使用できます


Pythonのバージョン： 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
---------
PyTorchのバージョン： 2.5.1+cu124
GPUの枚数： 1
cuda:0


## [0-6] Google Driveのマウント（任意）


In [7]:
# from google.colab import drive
# drive.mount('/content/gdrive')

---

# [1] ひとまず適当にPendulum環境を動かす

## [1-1] Pendulum環境を作成

In [8]:
import gym

# version確認
print(gym.__version__)  # 0.25.2でした

# CartPoleのenvを用意
env = gym.make("Pendulum-v1")


0.25.2


## [1-2] 適当に動かす

In [9]:
# 適当に動かす
# ====================
# [0] 環境をリセット+初期変数の設定
state = env.reset()
done = False        # episodeの終わりを判定するフラグ
episode_reward = 0  # episodeの総報酬を格納する
frames = []         # 画像保存用リスト

# 200step経過して、done=Trueになるまで続ける
while not done:
    # [1] 画面キャプチャ
    frames.append(env.render(mode="rgb_array"))  # ndarray (400, 600, 3)

    # [2] Actionを適当に選ぶ（連続値-2から2）
    action = random.uniform(-2.0, 2.0)

    # [3] Actionを適用してstateを更新する
    state, reward, done, _ = env.step([action])
    episode_reward += reward

# 実行結果
print("episode_reward:", episode_reward)
env.close()

episode_reward: -1087.8307958860314


## [1-3] 実行結果を動画で可視化

In [10]:
import matplotlib.animation as animation
from matplotlib import rc

def show_animation(imgs):
    """動画表示する関数。引数 imgs は ndarray (400, 600, 3) のリストを想定"""
    rc("animation", html="jshtml")

    fig, ax = plt.subplots(1, 1, figsize=(5, 3))
    frames = []

    # テキストを初期化（step 数）
    text = ax.text(10, 20, "", fontsize=12, color="black")

    for i, img in enumerate(imgs):
        frame = [ax.imshow(img, animated=True)]  # 画像を描画
        frame.append(ax.text(10, 20, f"Step: {i+1}", animated=True))  # Step数表示
        frames.append(frame)

    ax.axis("off")

    ani = animation.ArtistAnimation(fig, frames, interval=100, blit=True)

    # 動画保存
    ani.save("pendulum_grpo.mp4", writer="ffmpeg")
    ani.save("pendulum_grpo.gif", writer="pillow")

    plt.close(fig)  # ここで閉じる

    return ani


In [11]:
# 可視化の実行
show_animation(frames)


ひとまず、適当に動きました。が棒が上方向に直立する状態にはありません。

そこで、状態（state）に応じて、行動を決めるように、「Policy ネットワーク」を訓練（強化学習）して、真上で立つ制御を目指します。

この強化学習に今回はGRPOを利用します

# [2] GRPOに必要な関数群を定義する

## [2-1] Policy Networkを定義

PolicyのNeural Network は 現在のstateを入力とし、Actionの連続値の確率分布のパラメータを出力します

In [179]:
class PolicyNet(torch.nn.Module):
    def __init__(self):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(3, 64*2)
        self.fc2 = torch.nn.Linear(64*2, 64*2)
        # 平均（μ）を出力する層
        self.mu_layer = torch.nn.Linear(64*2, 1)


    def forward(self, state):
        x = torch.nn.functional.tanh(self.fc1(state))
        x = torch.nn.functional.tanh(self.fc2(x))
        mu = self.mu_layer(x)

        std = torch.tensor(0.1, device=device)  # stdは常に0.1とする

        return mu, std


## [2-2] 1エピソードを実行し、各stepでのstateやactionなどを収集する関数を定義

GRPOの重要な点として、報酬はエピソードの各step tで密に与えられるのではなく、エピソードの終わりに総報酬のみが渡されるとする（報酬が疎）。


そのため、エピソード途中の各stepに対する各actionに対しての細かい評価は個別には行わない

In [186]:
def collect_trajectory(env, net):
    # [0] 環境をリセット+初期変数の設定
    state = env.reset()  # gym 0.25以降の仕様対応
    states, log_probs, chosen_actions = [], [], []  # ここに蓄積
    episode_reward = 0
    done = False

    for t in range(50):  # 本当は200 stepで自動打ち切りですが、計算時間省略のために50 stepで打ち切ることにします
        # [1] そのstepでの各情報を求める
        states.append(state)  # numpy.ndarray のまま保存

        # [2] ネットワークの出力を (μ, σ) と解釈
        state_tensor = torch.from_numpy(state).float()  # 修正
        logits = net(state_tensor.to(device))  # ネットワークの出力
        mu, sigma = logits[0], logits[1]  # 2つの出力（平均、標準偏差）

        # [3] 正規分布からサンプリング
        normal_dist = torch.distributions.Normal(mu, sigma)
        action = normal_dist.sample()
        log_prob = normal_dist.log_prob(action)  # 選択したアクションの対数確率

        # [4] 各種格納
        log_probs.append(log_prob.item())
        chosen_actions.append(action.item())

        # [5] action_sclaingを適用してstateを更新する
        action_np = np.array([action.item()])  # numpy の形に変換
        state, reward, done, _ = env.step(action_np)  # 修正
        episode_reward += reward

    # [6] 適当な数で割って正規化（スケール調整）
    normalized_reward = episode_reward / 1000.0

    return states, log_probs, chosen_actions, normalized_reward


## [2-3] GRPOに基づくAdvantage計算を定義

ここが、GRPO（Group Relative Policy Optimization）の真髄です。

PPOやActor-Critic 等であれば、価値関数を求める「状態価値関数 V(state)」のNNを使用して、Advantage（≒そのstateでの平均価値と実際の行動の価値の差）を求めますが、GRPOは報酬のみからAdvantageを計算します。

そのため、状態価値（Value）を求めるNeural Networkが不要です

In [187]:
def calc_advantages_with_grpo(trajectories):
    """trajectoriesから報酬のみを取り出し、各エピソードの報酬を標準化します"""
    rewards = [r for o, l, a, r in trajectories]  # 最終報酬を取り出して、
    mean_reward = sum(rewards) / len(rewards)     # 平均値求めて、
    std_reward = np.std(rewards)  + 1e-8          # 標準偏差求めて（1e-8は0除算対策）、
    advantages = [(r - mean_reward) / std_reward for r in rewards]  # 最後に各エピソードについて標準化

    return advantages


## [2-4] GRPOに基づくPolicy Netの重み更新関数を定義

注意：GRPOはPPOの更新式とは少し異なり、本当はKLダイバージェンスで過度な更新を防ぐ項が入りますが、今回の実装ではその項は省略しています

In [191]:
def grpo_update(trajectories, net, optimizer, n_iterations=10, eps=0.2):

    # [1] [2-3]のGRPOの関数で、各エピソードの標準化されたAdvantageを求めます
    advantages = calc_advantages_with_grpo(trajectories)

    # [2] Policy NN を更新します。n_iterations回、更新をかけます
    for i_iter in range(n_iterations):
        loss = 0
        for traj, advantage in zip(trajectories, advantages):
            (states, log_probs, chosen_actions, _) = traj  # 1エピソードの蓄えられた内容を取り出します
            trajectory_loss = 0                            # 1エピソードの損失を0に初期化

            # [3] エピソード内の各ステップについて損失（loss）を計算
            for t in range(len(states)):
                # 以下はPPOと同じ手続きです
                # ====================================
                state_tensor = torch.from_numpy(states[t]).float()  # PyTorch Tensor に変換
                action_tensor = torch.tensor(chosen_actions[t]).float()  # Action も Tensor にする

                # [3-1] 更新された Policy NN で新しい行動分布を求める
                logits = net(state_tensor.to(device))
                mu, sigma = logits[0], logits[1]  # ネットワークの出力 (μ, σ)

                # 正規分布を作成し、行動の log_prob を計算
                new_dist = torch.distributions.Normal(mu, sigma)
                new_log_prob = new_dist.log_prob(action_tensor)  # 選択された action の log 確率

                # [3-2] 更新前後の log 確率の比率を計算
                ratio = torch.exp(new_log_prob - log_probs[t])

                # [3-3] 更新後の選ばれる確率が非常に大きくなってしまうと更新しすぎなので、epsの範囲にとどめる
                clipped_ratio = torch.clamp(ratio, min=1 - eps, max=1 + eps)

                # [3-4] 報酬を最大化したいので、マイナスを掛け算して最小化問題に置き換える
                # そして平均より良かった施行を増やし、悪かったケースを減らすように更新
                trajectory_loss += -clipped_ratio * advantage

            # [4] エピソードのsteps数で正規化
            trajectory_loss /= len(states)
            loss += trajectory_loss

        # [5] エピソード数で正規化
        loss /= len(trajectories)

        # [6] Policy NN の重みを更新
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return None  # 特に何も返さない

# [3] GRPOでPolicy Netを更新しながら、Pendulumを実行する

GRPOを実行するための関数群が定義できたので、早速実行します

## [3-1] 初期化とハイパラ設定

In [192]:
# [1] 初期化と初期設定
env = gym.make("Pendulum-v1")
net = PolicyNet()
net = net.to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4)

# [2] GRPO で PocicyNetを更新する際のTrajectoryを溜める回数の設定
trajectories_per_update = 64  # group size。ほぼミニバッチに対応するため、大きな値に変更しています


## [3-2] 訓練開始

以下の実行は、1トライアルに2分弱かかります。

今回は29回終了時点で強制停止しました。

In [193]:
# [3] エピソードのループを回実施
trial_num = 0  # 何回tryしたかを示す

for trial in range(250):
    # [4] エピソードのループを開始
    for i_episode in range(2):  # trajectories_per_updateが64なので、2で128回になります
        trajectories, episode_rewards = [], []  # episode_rewardsには実際の総報酬を格納

        # [5] GRPO で PocicyNetを更新する際のTrajectoryを溜める
        for _ in range(trajectories_per_update):
            states, log_probs, chosen_actions, normalized_reward = collect_trajectory(env, net)
            trajectories.append((states, log_probs, chosen_actions, normalized_reward))
            episode_rewards.append(normalized_reward * 1000.0)  # 実際の総報酬を格納

        # [6] GRPO で PociyNetの重みを更新
        grpo_update(trajectories, net, optimizer)

    # [7] エピソードの平均報酬を求める
    avg_reward = sum(episode_rewards) / len(episode_rewards)
    trial_num+=1
    print(f'トライアル {trial_num}回目, avg reward: {avg_reward:.2f}')

env.close()


トライアル 1回目, avg reward: -328.68
トライアル 2回目, avg reward: -312.43
トライアル 3回目, avg reward: -302.13
トライアル 4回目, avg reward: -287.37
トライアル 5回目, avg reward: -295.16
トライアル 6回目, avg reward: -274.82
トライアル 7回目, avg reward: -260.32
トライアル 8回目, avg reward: -279.15
トライアル 9回目, avg reward: -271.46
トライアル 10回目, avg reward: -254.04
トライアル 11回目, avg reward: -225.59
トライアル 12回目, avg reward: -250.95
トライアル 13回目, avg reward: -256.74
トライアル 14回目, avg reward: -240.36
トライアル 15回目, avg reward: -226.38
トライアル 16回目, avg reward: -229.28
トライアル 17回目, avg reward: -224.45
トライアル 18回目, avg reward: -221.74
トライアル 19回目, avg reward: -192.53
トライアル 20回目, avg reward: -203.70
トライアル 21回目, avg reward: -244.95
トライアル 22回目, avg reward: -208.40
トライアル 23回目, avg reward: -244.62
トライアル 24回目, avg reward: -206.06
トライアル 25回目, avg reward: -258.42
トライアル 26回目, avg reward: -222.92
トライアル 27回目, avg reward: -224.21
トライアル 28回目, avg reward: -235.68
トライアル 29回目, avg reward: -224.41


KeyboardInterrupt: 

# [4] 訓練後のPolicy NetでPendulumを制御する様子を可視化する

## [4-1] 1エピソードの実施

In [196]:
# [0] 環境をリセット+初期変数の設定
state = env.reset()
done = False        # episodeの終わりを判定するフラグ
episode_reward = 0  # episodeの総報酬を格納する
frames = []         # 画像保存用リスト

# 200step経過して、done=Trueになるまで続ける
while not done:
    # [1] 画面キャプチャ
    frames.append(env.render(mode="rgb_array"))  # ndarray (400, 600, 3)

    # [2] ActionをPolicyNetから求める：ネットワークの出力を (μ, σ) と解釈
    state_tensor = torch.from_numpy(state).float()  # 修正
    logits = net(state_tensor.to(device))  # ネットワークの出力
    mu, sigma = logits[0], logits[1]  # 2つの出力（平均、標準偏差）

    # [3] 正規分布からサンプリング
    normal_dist = torch.distributions.Normal(mu, sigma)
    action = normal_dist.sample()
    log_prob = normal_dist.log_prob(action)  # 選択したアクションの対数確率

    # [4] Actionを適用してstateを更新する
    action_np = np.array([action.item()])  # numpy の形に変換
    state, reward, done, _ = env.step(action_np)  # 修正
    episode_reward += reward

# 実行結果
print("episode_reward:", episode_reward)
env.close()

episode_reward: -236.63611280526


## [4-2] 実行結果を動画で可視化

In [197]:
# 可視化
show_animation(frames)


# 以上